In [ ]:
import jupyter_black

jupyter_black.load()
import pandas as pd
import nfl_data_py as nfl
from src.visualisation.visualisation import plot_bar
from src.visualisation import colors
import plotly.express as px

pd.set_option("display.max_columns", None)

In [ ]:
pbp: pd.DataFrame = nfl.import_pbp_data([2023])

In [ ]:
# offense_players - list of id's
id_df = nfl.import_ids()
id_df.sample(4)

In [ ]:
pbp = pbp.query("week<=18").query('play_type=="pass" or play_type=="run"')

In [ ]:
pbp["offense_players_list"] = pbp["offense_players"].apply(
    lambda x: x.split(";") if type(x) == str else None
)

In [ ]:
pbp["offense_players_list"]

In [ ]:
pbp_player = (
    pbp.explode("offense_players_list")
    .dropna(subset=["offense_players_list"])
    .rename(columns={"offense_players_list": "player_id"})
    # .query('player_id!=""')
    .merge(
        id_df[["gsis_id", "position", "name"]],
        how="left",
        left_on="player_id",
        right_on="gsis_id",
    )
    .drop(columns=["gsis_id"])
)

In [ ]:
pbp_player.sample(8)

In [ ]:
pbp_wr = pbp_player.query('position=="WR"').copy()
pbp_wr["own_target"] = pbp_wr.apply(
    lambda row: 1 if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)
pbp_wr["own_rec"] = pbp_wr.apply(
    lambda row: row["complete_pass"] if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)
pbp_wr["own_rec_yds"] = pbp_wr.apply(
    lambda row: row["receiving_yards"] if row["player_id"] == row["receiver_id"] else 0,
    axis=1,
)

In [ ]:
arsb = pbp_wr.query("player_id=='00-0036963'")

In [ ]:
# 00-0030279 keenan
# 00-0036963 arsb

In [ ]:
[c for c in arsb.columns if "rec" in c]

In [ ]:
arsb.head()

In [ ]:
yrr_df = (
    pbp_wr.groupby(["player_id", "name_y", "posteam"])
    .agg(
        {
            "play_id": "count",
            "pass_attempt": "sum",
            "own_target": "sum",
            "own_rec": "sum",
            "own_rec_yds": "sum",
        }
    )
    .rename(
        columns={
            "play_id": "snaps",
            "pass_attempt": "routes",
            "own_target": "tgts",
            "own_rec": "recs",
            "own_rec_yds": "yds",
        }
    )
    .astype(int)
)
yrr_df["yrr"] = (yrr_df["yds"] / yrr_df["routes"]).round(3)
yrr_df = yrr_df.query("routes>50").sort_values("yrr", ascending=False)
yrr_df.head(25)

In [ ]:
yrr_plot = (
    yrr_df.reset_index()
    .head(20)[["name_y", "posteam", "yrr"]]
    .rename(columns={"name_y": "player", "posteam": "team"})
    .sort_values("yrr", ascending=True)
)
yrr_plot

In [ ]:
plot_bar(yrr_plot.head(20), x="yrr", y="player", dark_mode=False)

In [ ]:
fig = px.bar(
    yrr_plot,
    x="yrr",
    y="player",
    orientation="h",
    color="team",
    color_discrete_map=colors.team_unique_colors,
    opacity=0.85,
)
# Iterate through the data and add logos to the chart
for index, row in yrr_plot.iterrows():
    scale = 1.25
    fig.add_layout_image(
        dict(
            source=f"https://a.espncdn.com/i/teamlogos/nfl/500/{row['team']}.png",
            x=row["yrr"],  # Adjust the position
            y=row["player"],
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",
        )
    )
fig.update_yaxes(categoryorder="total ascending")
MARGIN = 100
fig.update_layout(
    height=800,
    width=700,
    template="plotly_dark",
    # template="plotly_white",
    xaxis_title=f"Yards per Route Run",
    yaxis_title="",
    title=f"2023 Yards per Route Run",
    title_x=0.5,  # Centers the title horizontally
    annotations=[
        dict(
            text="Data: @nflfastR | Chart: @griffreichert",
            x=-0.25,  # centered
            y=-0.1,  # position above
            xref="paper",
            yref="paper",
            showarrow=False,
            xanchor="left",
            yanchor="top",
            font=dict(size=12),
        )
    ],
    margin=dict(
        t=MARGIN, b=MARGIN, l=MARGIN, r=MARGIN
    ),  # Adjust top and bottom margin to make space for the title and the note
)
fig.update_traces(showlegend=False)
# Show the chart
fig.show()